In [ ]:
!pip install transformers==2.11.0
!pip install nlp==0.2.0
!pip install datasets

     |████████████████████████████████| 675kB 7.3MB/s 
     |████████████████████████████████| 5.6MB 15.1MB/s 
     |████████████████████████████████| 1.2MB 36.1MB/s 
     |████████████████████████████████| 901kB 50.9MB/s 
     |████████████████████████████████| 860kB 7.3MB/s 
     |████████████████████████████████| 225kB 9.0MB/s 
     |████████████████████████████████| 245kB 12.2MB/s 
     |████████████████████████████████| 112kB 13.9MB/s 


In [ ]:
import transformers
import torch
import numpy as np
import torch.nn as nn
import nlp
from datasets import load_dataset
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
dataset_dict={
        'stsb':load_dataset('glue',name='stsb'),
        'rte':load_dataset('glue',name='rte'),}

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/rte/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [ ]:
for task_name,dataset in dataset_dict.items():
  print(task_name)
  print(dataset_dict[task_name]['train'][0])
  print()

stsb
{'idx': 0, 'label': 5.0, 'sentence1': 'A plane is taking off.', 'sentence2': 'An air plane is taking off.'}

rte
{'idx': 0, 'label': 1, 'sentence1': 'No Weapons of Mass Destruction Found in Iraq Yet.', 'sentence2': 'Weapons of Mass Destruction Found in Iraq.'}



In [ ]:
class MultitaskModel(transformers.PreTrainedModel):
  def __init__(self,encoder,taskmodels_dict):
    super().__init__(transformers.PretrainedConfig())

    self.encoder=encoder
    self.taskmodels_dict=nn.ModuleDict(taskmodels_dict)

  @classmethod
  def create(cls,model_name,model_type_dict,model_config_dict):
    shared_encoder=None
    taskmodels_dict={}
    for task_name,model_type in model_type_dict.items():
      model=model_type.from_pretrained(model_name,config=model_config_dict[task_name]) 

      if shared_encoder is None:
        shared_encoder=getattr(model,cls.get_encoder_attr_name(model))
      else:
        setattr(model,cls.get_encoder_attr_name(model),shared_encoder)
      taskmodels_dict[task_name]=model

    return cls(encoder=shared_encoder,taskmodels_dict=taskmodels_dict)

  @classmethod
  def get_encoder_attr_name(cls,model):
    model_class_name=model.__class__.__name__
    if model_class_name.startswith('Bert'):
      return 'bert'
    elif model_class_name.startswith('Roberta'):
      return 'roberta'
    elif model_class_name.startswith('Albert'):
      return 'albert'
    else:
      raise KeyError(F'Add support for new model {model_class_name}')

  def forward(self,task_name,**kwargs):
    return self.taskmodels_dict[task_name](**kwargs)              

In [ ]:
model_name='roberta-base'
multitask_model=MultitaskModel.create(
    model_name=model_name,
    model_type_dict={
        'stsb':transformers.AutoModelForSequenceClassification,
        'rte':transformers.AutoModelForSequenceClassification,
    },
    model_config_dict={
        'stsb':transformers.AutoConfig.from_pretrained(model_name,num_labels=1),
        'rte':transformers.AutoConfig.from_pretrained(model_name,num_labels=2),
    }
)

INFO:filelock:Lock 140443642500240 acquired on /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmptk_p2tpp


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json in cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
INFO:filelock:Lock 140443642500240 released on /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690.lock
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
INFO:transformers.configuration_utils:Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 50265
}

INFO:filelock:Lock 140443642294992 acquired on /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a

INFO:transformers.file_utils:storing https://cdn.huggingface.co/roberta-base-pytorch_model.bin in cache at /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e
INFO:filelock:Lock 140443642294992 released on /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e.lock
INFO:transformers.modeling_utils:loading weights file https://cdn.huggingface.co/roberta-base-pytorch_model.bin from cache at /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e


INFO:transformers.modeling_utils:Weights of RobertaForSequenceClassification not initialized from pretrained model: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
INFO:transformers.modeling_utils:Weights from pretrained model not used in RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
INFO:transformers.modeling_utils:loading weights file https://cdn.huggingface.co/roberta-base-pytorch_model.bin from cache at /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e
INFO:transformers.modeling_utils:Weights of RobertaForSequenceClassification not initialized from pretrained model: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
INFO

In [ ]:
print(multitask_model)

MultitaskModel(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05

In [ ]:
print(multitask_model.encoder.embeddings.word_embeddings.weight.data_ptr())
print(multitask_model.taskmodels_dict['stsb'].roberta.embeddings.word_embeddings.weight.data_ptr())
print(multitask_model.taskmodels_dict['rte'].roberta.embeddings.word_embeddings.weight.data_ptr())

94618320199680
94618320199680
94618320199680


In [ ]:
tokenizer=transformers.AutoTokenizer.from_pretrained(model_name)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
INFO:transformers.configuration_utils:Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 50265
}

INFO:filelock:Lock 140443620895888 acquired on /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json in cache at /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
INFO:filelock:Lock 140443620895888 released on /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock


INFO:filelock:Lock 140443622193744 acquired on /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpjd85184y


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt in cache at /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:filelock:Lock 140443622193744 released on /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json from cache at /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d51427

In [ ]:
max_length=128

def convert_to_stsb_features(example_batch):
  inputs=list(zip(example_batch['sentence1'],example_batch['sentence2']))
  features=tokenizer.batch_encode_plus(inputs,max_length=max_length,
                                       pad_to_max_length=True)
  features['labels']=example_batch['label']
  return features

def convert_to_rte_features(example_batch):
  inputs=list(zip(example_batch['sentence1'],example_batch['sentence2']))
  features=tokenizer.batch_encode_plus(inputs,max_length=max_length,
                                       pad_to_max_length=True)
  features['labels']=example_batch['label']
  return features   

In [ ]:
convert_func_dict={
    'stsb':convert_to_stsb_features,
    'rte':convert_to_rte_features,
}

In [ ]:
columns_dict={
    'stsb':['input_ids','attention_mask','labels'],
    'rte':['input_ids','attention_mask','labels'],
}

In [ ]:
for task_name,dataset in dataset_dict.items():
  for phase,phase_dataset in dataset.items():
    print(phase)

train
validation
test
train
validation
test


In [ ]:
features_dict={}
for task_name,dataset in dataset_dict.items():
  features_dict[task_name]={}
  for phase,phase_dataset in dataset.items():
    features_dict[task_name][phase]=phase_dataset.map(convert_func_dict[task_name],
                                                      batched=True,
                                                      load_from_cache_file=False,)
    print(task_name,len(phase_dataset),len(features_dict[task_name][phase]))
    features_dict[task_name][phase].set_format(
        type='torch',
        columns=columns_dict[task_name],)
    print(task_name,len(phase_dataset),len(features_dict[task_name][phase]))
    


stsb 5749 5749
stsb 5749 5749



stsb 1500 1500
stsb 1500 1500



stsb 1379 1379
stsb 1379 1379



rte 2490 2490
rte 2490 2490



rte 277 277
rte 277 277



rte 3000 3000
rte 3000 3000


In [ ]:
import dataclasses
from torch.utils.data.dataloader import DataLoader
from transformers.training_args import is_tpu_available
from transformers.trainer import get_tpu_sampler
from transformers.data.data_collator import DataCollator,InputDataClass
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data.sampler import RandomSampler
from typing import List,Union,Dict

In [ ]:
class NLPDataCollator(DataCollator):

  def collate_batch(self,features:List[Union[InputDataClass,Dict]]) ->Dict[str,torch.Tensor]:
     first=features[0]
     #print(first)
     if isinstance(first,dict):
        if 'labels' in first and first['labels'] is not None:
          if first['labels'].dtype==torch.int64:
            labels=torch.tensor([f['labels']for f in features],dtype=torch.long)
          else:
            labels=torch.tensor([f['labels'] for f in features],dtype=torch.float)
          batch={'labels':labels}
        for k,v in first.items():
          if k!='labels' and v is not None and not isinstance(v,str):
            batch[k]=torch.stack([f[k] for f in features])
            #print('shape:',batch[k].shape)
            #print('batch:',batch[k]) 
        return batch
     else:
       return DefaultDataCollator().collate_batch(features)  

In [ ]:
features_dict['stsb']['train'][0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([   0,   83, 3286,   16,  602,  160,    4,    2,    2,  660,  935, 3286,
           16,  602,  160,    4,    2,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1

In [ ]:
print(features_dict.keys())

dict_keys(['stsb', 'rte'])


In [ ]:
class StrIgnoreDevice(str):
  def to(self,device):
    return self

class DataLoaderWithTaskname:
  def __init__(self,task_name,data_loader):
    self.task_name=task_name
    self.data_loader=data_loader

    self.batch_size=data_loader.batch_size
    self.dataset=data_loader.dataset

  def __len__(self):
    return len(self.data_loader)

  def __iter__(self):
    for batch in self.data_loader:
      batch['task_name']=StrIgnoreDevice(self.task_name)
      yield batch  

In [ ]:
class MultitaskDataloader:
  def __init__(self,dataloader_dict):
    self.dataloader_dict=dataloader_dict
    self.num_batches_dict={
        task_name:len(dataloader)
        for task_name,dataloader in self.dataloader_dict.items()
    }
    self.task_name_list=list(self.dataloader_dict)
    self.dataset=[None]*sum(len(dataloader.dataset) for dataloader in self.dataloader_dict.values())

  def __len__(self):
    return sum(self.num_batches_dict.values()) 

  def __iter__(self):
    task_choice_list=[]
    for i,task_name in enumerate(self.task_name_list):
      task_choice_list+=[i]*self.num_batches_dict[task_name]
    task_choice_list=np.array(task_choice_list)
    np.random.shuffle(task_choice_list)
    dataloader_iter_dict={task_name:iter(dataloader)
                          for task_name,dataloader in self.dataloader_dict.items()} 
    for task_choice in task_choice_list:
      task_name=self.task_name_list[task_choice]
      yield next(dataloader_iter_dict[task_name])                       

In [ ]:
class MultitaskTrainer(transformers.Trainer):
  def get_single_train_dataloader(self,task_name,train_dataset):
    if self.train_dataset is None:
      raise ValueError('Trainer:training requires a train_dataset.')
    if is_tpu_available():
            train_sampler=get_tpu_sampler(train_dataset)
    else:
          train_sampler=(RandomSampler(train_dataset) if self.args.local_rank==-1 else DistributedSampler(train_dataset))

    data_loader=DataLoaderWithTaskname(task_name=task_name,
                                       data_loader=DataLoader(train_dataset,
                                                      batch_size=self.args.train_batch_size,
                                                      sampler=train_sampler,
                                                      collate_fn=self.data_collator.collate_batch,),)

    
    return data_loader

  def get_train_dataloader(self):
    return MultitaskDataloader({task_name:self.get_single_train_dataloader(task_name,task_dataset)
                               for task_name,task_dataset in self.train_dataset.items()})                        


In [ ]:
train_dataset={task_name:dataset['train'] 
               for task_name,dataset in features_dict.items()}

In [ ]:
trainer=MultitaskTrainer(model=multitask_model,
                         args=transformers.TrainingArguments(
                             output_dir='./models/multitask_model',
                             overwrite_output_dir=True,
                             learning_rate=1e-5,
                             do_train=True,
                             num_train_epochs=3,
                             per_device_train_batch_size=8,
                             save_steps=3000),
                         data_collator=NLPDataCollator(),
                         train_dataset=train_dataset,)

INFO:transformers.training_args:PyTorch: setting up devices
INFO:transformers.trainer:You are instantiating a Trainer but W&B is not installed. To use wandb logging, run `pip install wandb; wandb login` see https://docs.wandb.com/huggingface.


In [ ]:
trainer.train()

INFO:transformers.trainer:***** Running training *****
INFO:transformers.trainer:  Num examples = 8239
INFO:transformers.trainer:  Num Epochs = 3
INFO:transformers.trainer:  Instantaneous batch size per device = 8
INFO:transformers.trainer:  Total train batch size (w. parallel, distributed & accumulation) = 8
INFO:transformers.trainer:  Gradient Accumulation steps = 1
INFO:transformers.trainer:  Total optimization steps = 3093


In [ ]:
trainer.save_model('/content/drive/MyDrive/multi_task_model')

INFO:transformers.trainer:Saving model checkpoint to /content/drive/MyDrive/multi_task_model
INFO:transformers.configuration_utils:Configuration saved in /content/drive/MyDrive/multi_task_model/config.json
INFO:transformers.modeling_utils:Model weights saved in /content/drive/MyDrive/multi_task_model/pytorch_model.bin


In [ ]:
preds_dict={}
for task_name in ['rte','stsb']:
  eval_dataloader=DataLoaderWithTaskname(
      task_name,
      trainer.get_eval_dataloader(eval_dataset=features_dict[task_name]['validation']))
  print(eval_dataloader.data_loader.collate_fn)
  preds_dict[task_name]=trainer._prediction_loop(eval_dataloader,
                                                 description=f'Validation:{task_name}')  

INFO:transformers.trainer:***** Running Validation:rte *****
INFO:transformers.trainer:  Num examples = 277
INFO:transformers.trainer:  Batch size = 8


<bound method NLPDataCollator.collate_batch of <__main__.NLPDataCollator object at 0x7fbb956a3bd0>>


INFO:transformers.trainer:***** Running Validation:stsb *****
INFO:transformers.trainer:  Num examples = 1500
INFO:transformers.trainer:  Batch size = 8



<bound method NLPDataCollator.collate_batch of <__main__.NLPDataCollator object at 0x7fbb956a3bd0>>


In [ ]:
print(eval_dataloader.data_loader.collate_fn)

<bound method NLPDataCollator.collate_batch of <__main__.NLPDataCollator object at 0x7fbb956a3bd0>>


In [ ]:
#Evaluate RTE
nlp.load_metric('glue',name='rte').compute(
    np.argmax(preds_dict['rte'].predictions,axis=1),
    preds_dict['rte'].label_ids,
)

INFO:filelock:Lock 140443559476176 acquired on /root/.cache/huggingface/datasets/ee5b3a098be9a0d5be9e705b2abdaf1c7bf81ebf279e965db8dbd7db418efa32.f1fd3484ce65950de4cdde6c3e2f332d0fc7dd681ea11d91ede37857561b30b4.py.lock
INFO:nlp.utils.file_utils:https://s3.amazonaws.com/datasets.huggingface.co/nlp/metrics/glue/glue.py not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/tmp8_dulem6


INFO:nlp.utils.file_utils:storing https://s3.amazonaws.com/datasets.huggingface.co/nlp/metrics/glue/glue.py in cache at /root/.cache/huggingface/datasets/ee5b3a098be9a0d5be9e705b2abdaf1c7bf81ebf279e965db8dbd7db418efa32.f1fd3484ce65950de4cdde6c3e2f332d0fc7dd681ea11d91ede37857561b30b4.py
INFO:nlp.utils.file_utils:creating metadata file for /root/.cache/huggingface/datasets/ee5b3a098be9a0d5be9e705b2abdaf1c7bf81ebf279e965db8dbd7db418efa32.f1fd3484ce65950de4cdde6c3e2f332d0fc7dd681ea11d91ede37857561b30b4.py
INFO:filelock:Lock 140443559476176 released on /root/.cache/huggingface/datasets/ee5b3a098be9a0d5be9e705b2abdaf1c7bf81ebf279e965db8dbd7db418efa32.f1fd3484ce65950de4cdde6c3e2f332d0fc7dd681ea11d91ede37857561b30b4.py.lock


INFO:nlp.load:Checking /root/.cache/huggingface/datasets/ee5b3a098be9a0d5be9e705b2abdaf1c7bf81ebf279e965db8dbd7db418efa32.f1fd3484ce65950de4cdde6c3e2f332d0fc7dd681ea11d91ede37857561b30b4.py for additional imports.
INFO:filelock:Lock 140443599281296 acquired on /root/.cache/huggingface/datasets/ee5b3a098be9a0d5be9e705b2abdaf1c7bf81ebf279e965db8dbd7db418efa32.f1fd3484ce65950de4cdde6c3e2f332d0fc7dd681ea11d91ede37857561b30b4.py.lock
INFO:nlp.load:Creating main folder for metric https://s3.amazonaws.com/datasets.huggingface.co/nlp/metrics/glue/glue.py at /usr/local/lib/python3.7/dist-packages/nlp/metrics/glue
INFO:nlp.load:Creating specific version folder for metric https://s3.amazonaws.com/datasets.huggingface.co/nlp/metrics/glue/glue.py at /usr/local/lib/python3.7/dist-packages/nlp/metrics/glue/8e05e2fd41da255e1d729512a956f95cd909869a50ab5c8ac5ff2a060fbd2c68
INFO:nlp.load:Copying script file from https://s3.amazonaws.com/datasets.huggingface.co/nlp/metrics/glue/glue.py to /usr/local/lib/p

{'accuracy': 0.7364620938628159}

In [ ]:
# Evalute STS-B
nlp.load_metric('glue', name="stsb").compute(
    preds_dict["stsb"].predictions.flatten(),
    preds_dict["stsb"].label_ids,
)

INFO:filelock:Lock 140443520085072 released on /root/.cache/huggingface/metrics/glue/1.0.0/cache-glue-0.arrow.lock
INFO:nlp.load:Checking /root/.cache/huggingface/datasets/ee5b3a098be9a0d5be9e705b2abdaf1c7bf81ebf279e965db8dbd7db418efa32.f1fd3484ce65950de4cdde6c3e2f332d0fc7dd681ea11d91ede37857561b30b4.py for additional imports.
INFO:filelock:Lock 140443535102224 acquired on /root/.cache/huggingface/datasets/ee5b3a098be9a0d5be9e705b2abdaf1c7bf81ebf279e965db8dbd7db418efa32.f1fd3484ce65950de4cdde6c3e2f332d0fc7dd681ea11d91ede37857561b30b4.py.lock
INFO:nlp.load:Found main folder for metric https://s3.amazonaws.com/datasets.huggingface.co/nlp/metrics/glue/glue.py at /usr/local/lib/python3.7/dist-packages/nlp/metrics/glue
INFO:nlp.load:Found specific version folder for metric https://s3.amazonaws.com/datasets.huggingface.co/nlp/metrics/glue/glue.py at /usr/local/lib/python3.7/dist-packages/nlp/metrics/glue/8e05e2fd41da255e1d729512a956f95cd909869a50ab5c8ac5ff2a060fbd2c68
INFO:nlp.load:Found scr

{'pearson': 0.9007454513797072, 'spearmanr': 0.8998729956569115}

In [ ]:
print(preds_dict['stsb'])

PredictionOutput(predictions=array([[5.073307  ],
       [5.0888114 ],
       [4.9457426 ],
       ...,
       [1.2783217 ],
       [0.56902736],
       [0.7268369 ]], dtype=float32), label_ids=array([5.  , 4.75, 5.  , ..., 2.  , 0.  , 0.  ], dtype=float32), metrics={'eval_loss': 0.4901568328842838})
